<h1>爬取股價與成交量
<h5> 標的：00737 國泰AI+Robo<br>
網站：台灣證券交易所

In [9]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from random import random
from tqdm import tqdm

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}

option = webdriver.ChromeOptions()
#option.add_argument('--headless')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome(options=option)
driver.maximize_window()

In [10]:
url = 'https://www.twse.com.tw/zh/trading/historical/stock-day.html'
driver.get(url)
driver.implicitly_wait(5)

In [13]:
# 創建一個空的 DataFrame 來存儲合併的資料
combined_df = pd.DataFrame()

# 使用 tqdm 顯示進度條
for i in tqdm(range(1, 13), desc="Fetching data for each month"):
    # 點擊年份
    driver.find_element(By.CSS_SELECTOR, '#label0').click()
    driver.find_element(By.CSS_SELECTOR, '#label0 > option:nth-child(2)').click()

    # 點擊月份
    driver.find_element(By.CSS_SELECTOR, '#form > div > div.groups > div.group.date-select.M > span > select:nth-child(2)').click()
    driver.find_element(By.CSS_SELECTOR, f'#form > div > div.groups > div.group.date-select.M > span > select:nth-child(2) > option:nth-child({i})').click()

    sleep(1 + random() * 2)

    # 輸入股票代號
    driver.find_element(By.CSS_SELECTOR, '#label1').clear()
    driver.find_element(By.CSS_SELECTOR, '#label1').send_keys('00737')

    # 送出
    driver.find_element(By.CSS_SELECTOR, '#form > div > div.groups > div.submit > button').click()

    sleep(1 + random() * 2)

    # 獲取網頁內容並解析
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    
    # 獲取標題
    headers = [header.text for header in table.find_all('th')]
    
    # 獲取表格內容
    rows = table.find_all('tr')
    data = []
    for row in rows[1:]:  # 跳過標題行
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        data.append(cols)
    
    # 創建當月的 DataFrame
    month_df = pd.DataFrame(data, columns=headers)
    
    # 合併到主 DataFrame
    combined_df = pd.concat([combined_df, month_df], ignore_index=True)

# 顯示結果
print(combined_df)

# 保存合併後的資料到新檔案
combined_df.to_csv('/Users/lomingyi/Downloads/期末專案/股價資訊/00737股價資訊_2023.csv', index=False)

# 關閉瀏覽器
driver.quit()

<h1>爬取 漲跌幅 資料
<h5>進入以下網址，往下滑到漲跌幅表格，匯出後手動刪除不必要的欄位<br>
https://goodinfo.tw/tw/ShowK_ChartCompare.asp?STOCK_ID=00737&STOCK_ID1=

<h1>爬取 Google Trend 資料
<h5>用 API（pytrend）很容易會被 google trend 限制存取，導致沒辦法抓到資料。<br>
所以最後改用手動輸入並下載<br>
網址：https://trends.google.com.tw/trends?geo=TW&hl=zh-TW

關鍵詞：AI、chatgpt、輝達、台積電

<h1>爬取 三大法人 資料
<h5>

In [2]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from random import random
from tqdm import tqdm

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}

option = webdriver.ChromeOptions()
#option.add_argument('--headless')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome(options=option)
driver.maximize_window()

In [3]:
url = 'https://www.twse.com.tw/zh/trading/foreign/t86.html'
driver.get(url)
driver.implicitly_wait(5)

In [ ]:
# 創建一個空的 DataFrame 並設置 headers
headers = []  # 初始化 headers 列表

# 使用 tqdm 顯示進度條
for i in tqdm(range(1, 13), desc="Fetching data for each month"):
    for j in tqdm(range(1, 32), desc="Fetching data for each day"):
        try:
            # 點擊年份
            driver.find_element(By.CSS_SELECTOR, '#label0').click()
            driver.find_element(By.CSS_SELECTOR, '#label0 > option:nth-child(2)').click()
            # 點擊月份
            driver.find_element(By.CSS_SELECTOR, '#form > div > div.groups > div.group.date-select.D > span > select:nth-child(2)').click()
            driver.find_element(By.CSS_SELECTOR, f'#form > div > div.groups > div.group.date-select.D > span > select:nth-child(2) > option:nth-child({i})').click()
            # 點擊日期
            driver.find_element(By.CSS_SELECTOR, '#form > div > div.groups > div.group.date-select.D > span > select:nth-child(3)').click()
            driver.find_element(By.CSS_SELECTOR, f'#form > div > div.groups > div.group.date-select.D > span > select:nth-child(3) > option:nth-child({j})').click()
            # 點擊查詢
            driver.find_element(By.CSS_SELECTOR, '#form > div > div.groups > div.submit > button').click()
            # 點擊類別
            driver.find_element(By.CSS_SELECTOR, '#label1').click()
            driver.find_element(By.CSS_SELECTOR, '#label1 > option:nth-child(4)').click()

            sleep(0.5 + random() * 2)

            # 檢查表格是否存在
            try:
                message = driver.find_element(By.CSS_SELECTOR, '#reports > div.main-content > div.rwd-table.dragscroll.sortable.F2.R3_.active > div').text
                if "很抱歉，沒有符合條件的資料!" in message:
                    print(f"No data for {i}-{j}")
                    continue
            except:
                pass

            # 選取顯示全部
            driver.find_element(By.CSS_SELECTOR, '#reports > hgroup > div > div.per-page > select').click()
            driver.find_element(By.CSS_SELECTOR, '#reports > hgroup > div > div.per-page > select > option:nth-child(5)').click()

            # 獲取網頁內容並解析
            sleep(1)  # 等待頁面加載
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            table = soup.find('table')

            if table:
                # 如果 headers 還沒有建立在 df 中，就先抓下來並建立
                if not headers:
                    header_elements = table.find_all('th')[2:]  # 從第三個 column 開始，因為前兩個是代號和名稱
                    headers = [header.text.strip() for header in header_elements]
                    combined_df = pd.DataFrame(columns=headers)

                rows = table.find_all('tr')
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) > 0 and cols[0].text.strip() == '00737':
                        print(f'{i}-{j} success!')
                        data = [f'{i}-{j}'] + [col.text.strip() for col in cols[2:]] 
                        print(data)
                        temp_df = pd.DataFrame([data], columns=headers)
                        combined_df = pd.concat([combined_df, temp_df], ignore_index=True)
                        break  # 找到目標資料後跳出循環

            # 添加延遲以避免被封鎖
            sleep(0.5 + random() * 2)
        except Exception as e:
            print(f"An error occurred on {i}-{j}: {e}")
            continue

driver.quit()

# 保存合併後的資料到 CSV 文件
combined_df.to_csv('/Users/lomingyi/Downloads/期末專案/00737三大法人資訊_2023.csv', index=False)

<h1>爬取 大盤指數 資料
<h5>目前先爬取「加權指數」、「NASDAQ」、「S&P500」、「」

In [ ]:
# 加權指數
import yfinance as yf

data = yf.download('^TWII', start='2023-01-01', end='2023-12-31')
data.to_csv('/Users/lomingyi/Downloads/期末專案/加權指數_2023.csv')
print('success!')

In [ ]:
# NASDAQ
import yfinance as yf

data = yf.download('^IXIC', start='2023-01-01', end='2023-12-31')
data.to_csv('/Users/lomingyi/Downloads/期末專案/NASDAQ_2023.csv')
print('success!')

In [ ]:
# S&P 500
import yfinance as yf

data = yf.download('^GSPC', start='2023-01-01', end='2023-12-31')
data.to_csv('/Users/lomingyi/Downloads/期末專案/SP500_2023.csv')
print('success!')

In [ ]:
# Dow Jones Industrial Average
import yfinance as yf

data = yf.download('^DJI', start='2023-01-01', end='2023-12-31')
data.to_csv('/Users/lomingyi/Downloads/期末專案/dowjones_2023.csv')
print('success!')